In [1]:
from pyspark.sql.functions import col, expr, when

df = spark.read.table("silver.cleaned_shopping_data")

# Encodage simplifie du type de visiteur
df_gold = df.withColumn("IsReturningVisitor", when(col("VisitorType") == "Returning_Visitor", 1).otherwise(0))

StatementMeta(, 7c699fe5-8425-4e7c-8675-8282de96b9f8, 3, Finished, Available, Finished)

In [11]:
# Creation de features de comportement
df_gold = df.withColumn("TotalPages", col("Administrative") + col("Informational") + col("ProductRelated")) \
            .withColumn("TotalDuration", col("Administrative_Duration") + col("Informational_Duration") + col("ProductRelated_Duration")) \
            .withColumn("PageValue_ExitRate_Ratio", col("PageValues") / (col("ExitRates") + 0.001))

StatementMeta(, d71e962d-d3fd-4df1-aab3-ffeea9a20255, 13, Finished, Available, Finished)

In [16]:
df_features = spark.read.table("gold.features_shopping")

#  Transformations 
df_features = df_features.withColumn("pages_per_minute", 
    when(col("TotalDuration") > 0, col("TotalPages") / (col("TotalDuration") / 60)).otherwise(0)) \
    .withColumn("is_peak_shopping_season", 
    when(col("Month").isin(['Nov', 'Dec', 'May']), 1).otherwise(0)) \
    .withColumn("high_intent_session", 
    when((col("PageValues") > 0) & (col("ExitRates") < 0.02), 1).otherwise(0))

StatementMeta(, d71e962d-d3fd-4df1-aab3-ffeea9a20255, 18, Finished, Available, Finished)

In [17]:
# Sauvegarde avec l'option de remplacement du schéma
df_features.write.mode("overwrite") \
    .format("delta") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold.features_shopping")

print(" Schéma mis à jour et données sauvegardées avec succès !")

StatementMeta(, d71e962d-d3fd-4df1-aab3-ffeea9a20255, 19, Finished, Available, Finished)

 Schéma mis à jour et données sauvegardées avec succès !


In [18]:
display(spark.read.table("gold.features_shopping").limit(5))

StatementMeta(, d71e962d-d3fd-4df1-aab3-ffeea9a20255, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1f91aaaf-ca83-47be-98a2-4ac5d761fad4)